# Sentinel-1 SAR Preprocessing Demo
## Hainan Naval Base - August 2025

This notebook demonstrates:
1. Loading Sentinel-1 SAR data
2. Converting to dB scale
3. Speckle filtering
4. Contrast enhancement
5. Visualization techniques

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.utils import load_sar_band, get_safe_info
from src.preprocessing import (
    convert_to_db, apply_speckle_filter, lee_filter,
    enhance_contrast, normalize_intensity
)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['figure.dpi'] = 100

print("✓ Modules loaded successfully")

## 1. Load SAR Data

In [ ]:
# Path to extracted SAFE directory
safe_dir = Path("../data/sentinel1/S1A_IW_GRDH_1SDV_20250816T025212_20250816T025237_060555_078800_1619.SAFE")

# Get product information
info = get_safe_info(safe_dir)
print("Product Information:")
print(f"  Satellite: {info['satellite']}")
print(f"  Product Type: {info['product_type']}")
print(f"  Start Time: {info['start_time']}")
print(f"  Polarizations: {info['polarizations']}")

In [ ]:
# Load VV polarization (better for ships and metal structures)
vv_data, vv_meta = load_sar_band(safe_dir, polarization='VV')

print(f"VV Data Shape: {vv_data.shape}")
print(f"Data Type: {vv_data.dtype}")
print(f"Value Range: {vv_data.min()} - {vv_data.max()}")
print(f"Mean: {vv_data.mean():.2f}")

## 2. Visualize Raw SAR Data

In [ ]:
# Take a subset for faster visualization
subset = vv_data[5000:7000, 10000:12000]

fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# Raw intensity
im1 = axes[0].imshow(subset, cmap='gray', vmin=0, vmax=np.percentile(subset, 99))
axes[0].set_title('Raw SAR Intensity (Linear Scale)', fontsize=14)
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)

# Histogram
axes[1].hist(subset.flatten(), bins=100, color='steelblue', alpha=0.7)
axes[1].set_xlabel('Intensity Value', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Intensity Histogram', fontsize=14)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Note: Raw SAR data shows high dynamic range and speckle noise")

## 3. Convert to dB Scale

In [ ]:
# Convert subset to dB
subset_db = convert_to_db(subset)

print(f"dB Range: {subset_db.min():.2f} to {subset_db.max():.2f} dB")

fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# dB scale image
im1 = axes[0].imshow(subset_db, cmap='gray', vmin=-25, vmax=5)
axes[0].set_title('SAR Data in dB Scale', fontsize=14)
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04, label='dB')

# dB histogram
axes[1].hist(subset_db.flatten(), bins=100, color='orange', alpha=0.7)
axes[1].set_xlabel('dB Value', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('dB Histogram', fontsize=14)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ dB scale provides better visualization and more normal distribution")

## 4. Speckle Filtering Comparison

In [ ]:
# Apply different filters
median_filtered = apply_speckle_filter(subset_db, method='median', size=5)
mean_filtered = apply_speckle_filter(subset_db, method='mean', size=5)
lee_filtered = lee_filter(subset, window_size=5)
lee_filtered_db = convert_to_db(lee_filtered)

fig, axes = plt.subplots(2, 2, figsize=(15, 15))

# Original
axes[0, 0].imshow(subset_db, cmap='gray', vmin=-25, vmax=5)
axes[0, 0].set_title('Original (dB)', fontsize=12)
axes[0, 0].axis('off')

# Median filter
axes[0, 1].imshow(median_filtered, cmap='gray', vmin=-25, vmax=5)
axes[0, 1].set_title('Median Filter (5x5)', fontsize=12)
axes[0, 1].axis('off')

# Mean filter
axes[1, 0].imshow(mean_filtered, cmap='gray', vmin=-25, vmax=5)
axes[1, 0].set_title('Mean Filter (5x5)', fontsize=12)
axes[1, 0].axis('off')

# Lee filter
axes[1, 1].imshow(lee_filtered_db, cmap='gray', vmin=-25, vmax=5)
axes[1, 1].set_title('Lee Filter (5x5)', fontsize=12)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Filter Performance:")
print(f"  Original variance: {np.var(subset_db):.2f}")
print(f"  Median filtered variance: {np.var(median_filtered):.2f}")
print(f"  Mean filtered variance: {np.var(mean_filtered):.2f}")
print(f"  Lee filtered variance: {np.var(lee_filtered_db):.2f}")

## 5. Contrast Enhancement

In [ ]:
# Apply different enhancement methods
eq_enhanced = enhance_contrast(median_filtered, method='equalize')
stretch_enhanced = enhance_contrast(median_filtered, method='stretch')

fig, axes = plt.subplots(1, 3, figsize=(20, 7))

# Original (filtered)
axes[0].imshow(median_filtered, cmap='gray', vmin=-25, vmax=5)
axes[0].set_title('Filtered (No Enhancement)', fontsize=14)
axes[0].axis('off')

# Histogram equalization
axes[1].imshow(eq_enhanced, cmap='gray')
axes[1].set_title('Histogram Equalization', fontsize=14)
axes[1].axis('off')

# Contrast stretch
axes[2].imshow(stretch_enhanced, cmap='gray')
axes[2].set_title('Contrast Stretch (2-98%)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("✓ Enhancement improves visual interpretation")

## 6. Full Processing Pipeline

In [ ]:
def process_sar_image(intensity_data, filter_size=5):
    """
    Complete SAR preprocessing pipeline.
    
    Steps:
    1. Convert to dB
    2. Apply median filter
    3. Enhance contrast
    4. Normalize to 0-1
    """
    # Convert to dB
    db_image = convert_to_db(intensity_data)
    
    # Speckle filtering
    filtered = apply_speckle_filter(db_image, method='median', size=filter_size)
    
    # Contrast enhancement
    enhanced = enhance_contrast(filtered, method='stretch')
    
    # Normalize
    normalized = normalize_intensity(enhanced)
    
    return normalized

# Process the subset
processed = process_sar_image(subset, filter_size=5)

# Visualize before and after
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(subset, cmap='gray', vmin=0, vmax=np.percentile(subset, 99))
axes[0].set_title('Raw SAR Data', fontsize=16, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(processed, cmap='gray')
axes[1].set_title('Processed SAR Data', fontsize=16, fontweight='bold')
axes[1].axis('off')

plt.suptitle('Sentinel-1 SAR Processing Pipeline', fontsize=18, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("✓ Processing pipeline complete")
print(f"  Output range: {processed.min():.3f} to {processed.max():.3f}")

## 7. Identify Features of Interest

In SAR imagery:
- **Bright pixels** = Strong backscatter (metal, buildings, ships)
- **Dark pixels** = Low backscatter (water, smooth surfaces)
- **Medium gray** = Moderate backscatter (concrete, soil)

In [ ]:
# Threshold to identify bright targets (ships, planes, buildings)
threshold = np.percentile(processed, 95)  # Top 5% brightest pixels

bright_targets = processed > threshold

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(processed, cmap='gray')
axes[0].set_title('Processed SAR Image', fontsize=14)
axes[0].axis('off')

axes[1].imshow(bright_targets, cmap='hot')
axes[1].set_title('Bright Targets (Potential Ships/Buildings)', fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"Detected {bright_targets.sum()} bright pixels (top 5%)")
print(f"This represents {100*bright_targets.sum()/bright_targets.size:.2f}% of the image")

## Summary

This notebook demonstrated:

✅ **Loading Sentinel-1 SAR data** from .SAFE archives  
✅ **Converting to dB scale** for better visualization  
✅ **Speckle filtering** using median, mean, and Lee filters  
✅ **Contrast enhancement** for improved interpretation  
✅ **Complete preprocessing pipeline** ready for analysis  

**Next Steps:**
- Multi-temporal change detection
- Object detection (ships, planes)
- Infrastructure analysis